In [1]:
import pandas as pd

In [2]:
crop_data = pd.read_csv('crop_yield_prediction.csv')
crop_data.head()

,Crop,Precipitation (mm day-1),Specific Humidity at 2 Meters (g/kg),Relative Humidity at 2 Meters (%),Temperature at 2 Meters (C),Yield
0,"Cocoa, beans",2248.92,17.72,83.40,26.01,11560
1,"Cocoa, beans",1938.42,17.54,82.11,26.11,11253
2,"Cocoa, beans",2301.54,17.81,82.79,26.24,9456
3,"Cocoa, beans",2592.35,17.61,85.07,25.56,9321
4,"Cocoa, beans",2344.72,17.61,84.12,25.76,8800


In [3]:
crop_data['Crop'].value_counts()

Crop
Cocoa, beans       39
Oil palm fruit     39
Rice, paddy        39
Rubber, natural    39
Name: count, dtype: int64

In [4]:
crop_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 6 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Crop                                  156 non-null    object 
 1   Precipitation (mm day-1)              156 non-null    float64
 2   Specific Humidity at 2 Meters (g/kg)  156 non-null    float64
 3   Relative Humidity at 2 Meters (%)     156 non-null    float64
 4   Temperature at 2 Meters (C)           156 non-null    float64
 5   Yield                                 156 non-null    int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 7.4+ KB


In [5]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [6]:
encoded_data = pd.get_dummies(crop_data)

In [7]:
len(encoded_data)

156

In [8]:
encoded_data.head()

,Precipitation (mm day-1),Specific Humidity at 2 Meters (g/kg),Relative Humidity at 2 Meters (%),Temperature at 2 Meters (C),Yield,"Crop_Cocoa, beans",Crop_Oil palm fruit,"Crop_Rice, paddy","Crop_Rubber, natural"
0,2248.92,17.72,83.40,26.01,11560,True,False,False,False
1,1938.42,17.54,82.11,26.11,11253,True,False,False,False
2,2301.54,17.81,82.79,26.24,9456,True,False,False,False
3,2592.35,17.61,85.07,25.56,9321,True,False,False,False
4,2344.72,17.61,84.12,25.76,8800,True,False,False,False


In [9]:
# Splitting the data into features and target
X = encoded_data.drop('Yield', axis=1).values
y = encoded_data['Yield'].values
X[:3]

array([[2248.92, 17.72, 83.4, 26.01, True, False, False, False],
       [1938.42, 17.54, 82.11, 26.11, True, False, False, False],
       [2301.54, 17.81, 82.79, 26.24, True, False, False, False]],
      dtype=object)

In [10]:
X.shape, y.shape

((156, 8), (156,))

In [11]:
# Splitting the datasets into the training and testing 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

In [12]:
# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [13]:
# Now predict using the linear regression model
y_prediction = lr.predict(X_test)
y_prediction[:5]

array([ 9054.09837034, 32703.63545354,  7464.75268134, 11005.12233152,
       34693.65943974])

In [14]:
# Now find metrics how our model is performing
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_prediction)
rmse = mse ** (1/2)
print(f"The root mean squared error by linear regression model: {rmse}")

The root mean squared error by linear regression model: 6139.91109633838


In [15]:
# Here we can also enhance the performance of our model by inserting regularization and also using cv
params = {'alpha': [0.0001, 0.001, 0.01, 0.1, 10, 100, 1000, 10000]}


In [16]:
las = Lasso()
search_cv = GridSearchCV(las, param_grid=params, cv=3)
search_cv.fit(X_train, y_train)

c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.869e+09, tolerance: 4.471e+07
  model = cd_fast.enet_coordinate_descent(
c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.576e+09, tolerance: 4.251e+07
  model = cd_fast.enet_coordinate_descent(
c:\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.940e+09, tolerance: 4.124e+07
  model = cd_fast.enet_

GridSearchCV(cv=3, estimator=Lasso(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 10, 100, 1000,
                                   10000]})

Finding the best parameter

In [17]:
search_cv.best_params_

{'alpha': 100}

In [18]:
# Use alpha = 100 to gain better efficiency
lasso = Lasso(alpha=100)
lasso.fit(X_train, y_train)

Lasso(alpha=100)

In [19]:
# precipation, humidity, relative_humidity, temperature, crop
# user_input = [precipation, humidity, relative_humidity, temperature, crop]
# crop = ['Crop Cocoa, beans',	'Crop Oil palm, fruit',	'Crop Rice, paddy',	'Crop Rubber, natural']  
#take this as an option for crop as option in html


# if crop.index == 0:
#     user_input.extend(1, 0, 0, 0)
# elif crop.index == 1:
#     user_input.extend(0, 1, 0, 0)
# elif crop.index == 2:
#     user_input.extend(0, 0, 1, 0)
# else:
#     user_input.extend(0, 0, 0, 1)

In [20]:
lasso_y_pred = lasso.predict(X_test)   #change here for the users output

In [21]:
lasso_y_pred

array([  9193.13625823,  31921.41807301,   7779.67503577,  11748.58002098,
        32803.72503343,  31450.11943715,  10534.75273511,   4677.56530907,
       176314.36908621, 175095.46083952,  30401.40795256,   8065.18406525,
         8232.84708961,   7843.12169068,   7170.8282842 , 174103.1150782 ,
        33758.4569269 ,  32762.30231689,   9372.18565113, 175263.12386388,
         9895.12017429,  12708.8691896 , 178160.5693164 ,  11130.29254213,
         8582.06039519,   7184.13560502,   7095.25054586,  32501.36465948,
         9814.77821126,   9462.55710406,   8411.8964825 ,   8595.45106203])

In [22]:
# finding the rmse to calculate whether the error is decreased or not
mse = mean_squared_error(y_test, lasso_y_pred)

print(f"The root mean squared error by lasso regression model: {mse ** (1/2)}")

The root mean squared error by lasso regression model: 6150.814341032196
